In [10]:
import torch
from typing import Optional
from extension_builder.builders.paged_attention import PagedAttentionBuilder
import random
from vllm.utils import get_max_shared_memory_bytes

import utils as test_utils

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
FLOAT32_BYTES = torch.finfo(torch.float).bits // 8
# This will change depending on the compute capability.
# - 512 as a buffer
MAX_SEQ_LEN = 2 ** 10 #get_max_shared_memory_bytes() // FLOAT32_BYTES - 512
NUM_BLOCKS = 40 #40000  # Arbitrary values for testing
PARTITION_SIZE = 512

DTYPES = [torch.half, torch.bfloat16, torch.float]
NUM_GEN_SEQS = [1]  # Arbitrary values for testing
NUM_PREFILL_SEQS = [3]  # Arbitrary values for testing
NUM_HEADS = [(4,4)]#[(40, 40), (64, 8)]  # Arbitrary values for testing
HEAD_SIZES = [32, 64, 80, 96, 112, 128, 256]
BLOCK_SIZES = [16, 32]
USE_ALIBI = [False, True]
SEEDS = [0]


In [3]:
num_seqs = NUM_GEN_SEQS[0]
num_heads = NUM_HEADS[0]
head_size = HEAD_SIZES[0]
use_alibi = USE_ALIBI[0]
block_size = BLOCK_SIZES[0]
dtype = DTYPES[0]
seed = SEEDS[0]
MAX_SEQ_LEN = MAX_SEQ_LEN
NUM_BLOCKS = NUM_BLOCKS
device = "cuda"


In [4]:
q, keys, values, ref_out = test_utils.run_test(num_seqs, num_heads, head_size, 
                                    use_alibi, block_size, dtype, 
                                    seed, MAX_SEQ_LEN, NUM_BLOCKS, 
                                    device=device)



In [8]:
lib = PagedAttentionBuilder().load()

/home/jeromeku/Cpp/CUDA/extension_builder/src/extension_builder/builders/utils.py:96: UserWarning: [extension] The CUDA version on the system (12.2) does not match with the version (12.1) torch was compiled with. The mismatch is found in the minor version. As the APIs are compatible, we will allow compilation to proceed. If you encounter any issue when using the built kernel, please try to build it again with fully matched CUDA versions
  warnings.warn(
The input conditions for extension module paged_attention have changed. Bumping to version 2 and re-building as paged_attention_v2...
Emitting ninja build file /home/jeromeku/.cache/extension_builder/torch_extensions/torch2.1_cu12.1/build.ninja...
Building extension module paged_attention_v2...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] c++ -MMD -MF attention_extension.o.d -DTORCH_EXTENSION_NAME=paged_attention_v2 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -I/home/jeromeku/Cpp/CUDA/extension_builder/csrc/ops/paged_attention -I/usr/local/cuda/include -isystem /home/jeromeku/miniconda3/envs/cutlass/lib/python3.10/site-packages/torch/include -isystem /home/jeromeku/miniconda3/envs/cutlass/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -isystem /home/jeromeku/miniconda3/envs/cutlass/lib/python3.10/site-packages/torch/include/TH -isystem /home/jeromeku/miniconda3/envs/cutlass/lib/python3.10/site-packages/torch/include/THC -isystem /home/jeromeku/miniconda3/envs/cutlass/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -O3 -c /home/jeromeku/Cpp/CUDA/extension_builder/csrc/ops/paged_attention/attention_extension.cpp -o attention_extension.o 
[2/3] c++ -MMD -MF attention.o.d -DTORCH_EXTENSION_NAME

Loading extension module paged_attention_v2...


In [15]:
args = test_utils.prepare_attention_inputs(
        num_seqs, num_heads, head_size, use_alibi, block_size, dtype, seed, 
        MAX_SEQ_LEN, NUM_BLOCKS, device=device
        )

output_ext = torch.empty_like(args.query)

In [16]:

res = lib.ref_single_query_cached_kv_attention(output_ext, 
                                               args.query, args.num_queries_per_kv, 
                                               args.key_cache, args.value_cache, 
                                               args.block_tables, args.context_lens,
                                               args.scale, args.alibi_slopes)

In [18]:
ext_q, ext_k, ext_v, ext_out = res

In [21]:
torch.allclose(ext_out[0], ref_out[0], atol=1e-4, rtol=1e-2)

True